In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/obesity-prediction/Obesity prediction.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [3]:
# load dataset

df = pd.read_csv("/kaggle/input/obesity-prediction/Obesity prediction.csv")

In [4]:
# label Encoding (categoric datas -> numeric)

label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

In [5]:
X = df.drop(columns=['Obesity'])
y = df.Obesity

In [6]:
# train test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# standardization

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
# classification models

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(kernel='linear', random_state=42)
}

In [9]:
# training and classification_report results

class_labels = label_encoders["Obesity"].classes_

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    print(f"Classification Report for {name}:\n")
    y_pred = model.predict(X_test_scaled)
    classification_results = classification_report(y_test, y_pred, target_names=class_labels)
    print(classification_results)

Classification Report for Logistic Regression:

                     precision    recall  f1-score   support

Insufficient_Weight       0.84      1.00      0.91        56
      Normal_Weight       0.91      0.63      0.74        62
     Obesity_Type_I       0.93      0.90      0.92        78
    Obesity_Type_II       0.89      0.97      0.93        58
   Obesity_Type_III       1.00      1.00      1.00        63
 Overweight_Level_I       0.77      0.79      0.78        56
Overweight_Level_II       0.76      0.84      0.80        50

           accuracy                           0.87       423
          macro avg       0.87      0.87      0.87       423
       weighted avg       0.88      0.87      0.87       423

Classification Report for Random Forest:

                     precision    recall  f1-score   support

Insufficient_Weight       0.98      0.96      0.97        56
      Normal_Weight       0.89      0.90      0.90        62
     Obesity_Type_I       0.99      0.97      0.98  

In [10]:
# PCA

pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# logistic regression after PCA
pca_classifier = LogisticRegression(max_iter=1000)
pca_classifier.fit(X_train_pca, y_train)
y_pred_pca = pca_classifier.predict(X_test_pca)

# results after PCA
pca_results = classification_report(y_test, y_pred_pca, target_names=class_labels)

print(pca_results)

                     precision    recall  f1-score   support

Insufficient_Weight       0.45      0.59      0.51        56
      Normal_Weight       0.31      0.26      0.28        62
     Obesity_Type_I       0.16      0.19      0.17        78
    Obesity_Type_II       0.49      0.76      0.60        58
   Obesity_Type_III       0.22      0.16      0.19        63
 Overweight_Level_I       0.17      0.18      0.17        56
Overweight_Level_II       0.00      0.00      0.00        50

           accuracy                           0.30       423
          macro avg       0.26      0.31      0.27       423
       weighted avg       0.26      0.30      0.27       423

